In [130]:
import pandas as pd
import numpy as np
import string as s
import seaborn as sns
import math
# import tables
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style, colors
from itertools import groupby
import matplotlib.patches as patches
from matplotlib import gridspec
import scipy.stats as stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, fclusterdata, leaves_list
from scipy.spatial.distance import pdist
from scipy.ndimage.interpolation import shift
# matplotlib.style.use('ggplot')
# %matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
# matplotlib.rcParams['figure.facecolor'] = 'white'
import mpl_toolkits.mplot3d.axes3d as p3
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets.samples_generator import make_swiss_roll
matplotlib.style.use('ggplot')
%matplotlib inline



In [131]:
wkdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/'

get meta data about library ids, hiv status, histology, total reads sequenced etc

In [132]:
# f = '/home/szong/projects/META_2018/Master_Sample_table_14X292_Feb27_2018_sz.xls'
f = '/home/szong/projects/META_2018/Master_Sample_table_14X292_Sep28_2018_sz.xls'
meta = pd.read_excel(f, sheet_name='HTMCP CaCx epi')
meta.head(2)

,billed status,Order,Biospecimen ID,HIV status,Epigenome modifier mutation status,H3K4me1 library ID,H3K4me3 library ID,H3K9me3 library ID,H3K27me3 library ID,H3K36me3 library ID,...,H3K4me3 % aligned,H3K9me3 % aligned,H3K27me3 % aligned,H3K36me3 % aligned,H3K27ac % aligned,input DNA control % aligned,pool ID,Notes,# lanes 75 PE,Internal sample ID
0,billed May 2018,1.0,HTMCP-03-06-02007-01A,negative,mutated,A84624,A84625,A84626,A84627,A84628,...,98.13,92.83,97.56,98.14,98.10,95.87,IX5978; IX6429,NaN,1.5,HTMCP_130
1,billed May 2018,2.0,HTMCP-03-06-02026-01A,negative,mutated,A84645,A84646,A84647,A84648,A84649,...,97.68,93.76,97.66,98.09,97.86,95.25,IX5981; IX6430,NaN,1.5,HTMCP_139


In [133]:
meta.shape

(57, 32)

In [134]:
meta = meta[['Biospecimen ID', 'H3K4me1 library ID',
       'H3K4me3 library ID', 'H3K9me3 library ID', 'H3K27me3 library ID',
       'H3K36me3 library ID', 'H3K27ac library ID',
       'input DNA control  library ID',  'H3K4me1 total reads', 'H3K4me3  total reads',
       'H3K9me3 total reads', 'H3K27me3 total reads', 'H3K36me3 total reads',
       'H3K27ac total reads', 'input DNA control  total reads']]#.dropna()

In [135]:
meta.head(2)

,Biospecimen ID,H3K4me1 library ID,H3K4me3 library ID,H3K9me3 library ID,H3K27me3 library ID,H3K36me3 library ID,H3K27ac library ID,input DNA control library ID,H3K4me1 total reads,H3K4me3 total reads,H3K9me3 total reads,H3K27me3 total reads,H3K36me3 total reads,H3K27ac total reads,input DNA control total reads
0,HTMCP-03-06-02007-01A,A84624,A84625,A84626,A84627,A84628,A84629,A84630,127431532.0,60179836.0,123495836.0,149090470.0,196198296.0,80676254.0,156512468.0
1,HTMCP-03-06-02026-01A,A84645,A84646,A84647,A84648,A84649,A84650,A84651,168600066.0,74487212.0,165810484.0,159223668.0,135643234.0,84986034.0,145945272.0


In [136]:
meta['patient'] = meta['Biospecimen ID'].apply(lambda x: '-'.join(['HTMCP-03-06', x.split('-')[3]]))

In [137]:
meta = meta.drop('Biospecimen ID', axis=1)

In [138]:
meta.head(2)

,H3K4me1 library ID,H3K4me3 library ID,H3K9me3 library ID,H3K27me3 library ID,H3K36me3 library ID,H3K27ac library ID,input DNA control library ID,H3K4me1 total reads,H3K4me3 total reads,H3K9me3 total reads,H3K27me3 total reads,H3K36me3 total reads,H3K27ac total reads,input DNA control total reads,patient
0,A84624,A84625,A84626,A84627,A84628,A84629,A84630,127431532.0,60179836.0,123495836.0,149090470.0,196198296.0,80676254.0,156512468.0,HTMCP-03-06-02007
1,A84645,A84646,A84647,A84648,A84649,A84650,A84651,168600066.0,74487212.0,165810484.0,159223668.0,135643234.0,84986034.0,145945272.0,HTMCP-03-06-02026


In [139]:
meta.shape

(57, 15)

In [140]:
# add in hiv status and histology
f22 ='/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/mutation_load_clinic.txt'
df22 = pd.read_csv(f22, sep='\t')
df22 = df22[['patient', 'HIV_status', 'Putative_histology', 'strelka_mutations', 'HPV_status']]
# df22 = df22.set_index('patient')#.stack().reset_index()
df22.head(2)

,patient,HIV_status,Putative_histology,strelka_mutations,HPV_status
0,HTMCP-03-06-02001,Positive,Squamous,26974,Positive
1,HTMCP-03-06-02002,Negative,Squamous,16661,Positive


In [141]:
df22.Putative_histology.unique()

array(['Squamous', 'Adeno'], dtype=object)

# get latest clinic histology

In [142]:
f33 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/mutations_for_oncoprint_123_patients.txt.new'
df33 = pd.read_csv(f33)
df44 = df33.iloc[:2,5:].T
df44.head()

,0,1
HTMCP-03-06-02007,Negative,Adenosquamous
HTMCP-03-06-02075,Negative,Adeno
HTMCP-03-06-02203,Negative,Adenosquamous
HTMCP-03-06-02047,Negative,Neuroendocrine
HTMCP-03-06-02026,Negative,Adeno


In [143]:
df55 = pd.merge(df22, df44, left_on='patient', right_index=True, how='left')

In [144]:
df55.head(2)
df55.shape

(123, 7)

In [145]:
df55[(df55.Putative_histology != df55[1])]

,patient,HIV_status,Putative_histology,strelka_mutations,HPV_status,0,1
4,HTMCP-03-06-02007,Negative,Adeno,2958282,Negative,Negative,Adenosquamous
15,HTMCP-03-06-02047,Negative,Adeno,8834,Positive,Negative,Neuroendocrine
36,HTMCP-03-06-02110,Positive,Adeno,8565,Positive,Positive,Adenosquamous
50,HTMCP-03-06-02176,Negative,Squamous,25803,Positive,Negative,Adenosquamous
55,HTMCP-03-06-02203,Negative,Adeno,35995,Positive,Negative,Adenosquamous
67,HTMCP-03-06-02216,Positive,Squamous,5831,Positive,Positive,Adenosquamous
76,HTMCP-03-06-02235,Positive,Adeno,7797,Positive,Positive,Neuroendocrine


In [146]:
df66 = df55.drop([0, 'Putative_histology'], axis=1)

In [147]:
df66.head(2)

,patient,HIV_status,strelka_mutations,HPV_status,1
0,HTMCP-03-06-02001,Positive,26974,Positive,Squamous
1,HTMCP-03-06-02002,Negative,16661,Positive,Squamous


In [148]:
df66.columns = df66.columns[:4].tolist() + ['clinic_histology']
df66.head(2)

,patient,HIV_status,strelka_mutations,HPV_status,clinic_histology
0,HTMCP-03-06-02001,Positive,26974,Positive,Squamous
1,HTMCP-03-06-02002,Negative,16661,Positive,Squamous


read in Alessia's epigenetic gene categorization file

# histone modifier complex

In [149]:
complexes = ['COMPASS complexes',
'Polycomb complexes',
'INO80/SWR remodelling complexes',
'ISWI remdelling complexes',
'SWI/SNF remodeling complex',
'SWI/SNF family',
'NuRD complex',
'HDAC complex',
'EP300/CBP HAT complexes',
'GNAT family HAT complexes',
'MYST family HAT complexes',
'HAT complexes',
'SEC (Super Elongation) Complex',
'TFIID basal transcription factor complex or PCAF histone acetylase complex',
'RNF20/40 complex']

In [150]:
# f4 = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/categorisation_of_epigentic_modifiers_from_Alessia_v3.xls'
f4 = '{0}{1}'.format(wkdir, 'categorisation_of_epigentic_modifiers_v4.xlsx')
df4 = pd.read_excel(f4, sheet_name='categorisation')
df4.head(2)

,HIV neg,neg samples with mutations,HIV positive,pos samples with mutation,total,category_1,category_2,markes /regiones affected,comments
0,ARID1A,2,ARID1A,1,3,Remodeller,SWI/SNF remodeling complex,NaN,NaN
1,ATRX,4,ATRX,1,5,Remodeller,SWI/SNF family,gene bodies of ZFN genes,Binds to zinc-finger coding genes with atypica...


In [151]:
df4[df4.category_2 == 'HDAC complex']

,HIV neg,neg samples with mutations,HIV positive,pos samples with mutation,total,category_1,category_2,markes /regiones affected,comments
39,HDAC10,1,HDAC10,0,1,Eraser,HDAC complex,H3K27ac,NaN
40,HDAC2,1,HDAC2,0,1,Eraser,HDAC complex,H3K27ac,NaN
42,HDAC4,0,HDAC4,3,3,Eraser,HDAC complex,H3K27ac,NaN
43,HDAC5,0,HDAC5,1,1,Eraser,HDAC complex,H3K27ac,NaN
44,HDAC6,1,HDAC6,2,3,Eraser,HDAC complex,H3K27ac,NaN
45,HDAC7,0,HDAC7,2,2,Eraser,HDAC complex,H3K27ac,NaN
46,HDAC9,3,HDAC9,0,3,Eraser,HDAC complex,H3K27ac,NaN


In [152]:
df41 = df4[['HIV neg', 'category_1', 'category_2']].set_index('HIV neg')
df41.head(2)

,category_1,category_2
HIV neg,,
ARID1A,Remodeller,SWI/SNF remodeling complex
ATRX,Remodeller,SWI/SNF family


In [153]:
df41.category_1.unique()

array(['Remodeller', 'Reader', 'Writer', nan, 'Eraser'], dtype=object)

In [154]:
df41.category_2.value_counts()

Polycomb complex                                                              10
INO80/SWR remodelling complexes                                                9
compass complex                                                                9
GNAT family HAT complexes                                                      7
ISWI remdelling complexes                                                      7
HDAC complex                                                                   7
SWI/SNF remodeling complex                                                     6
NuRD complex                                                                   5
MYST family HAT complexes                                                      5
SEC (Super Elongation) Complex                                                 4
EP300/CBP HAT complexes                                                        3
TFIID basal transcription factor complex or PCAF histone acetylase complex     3
chromatin remodelling       

In [155]:
df5 = pd.read_excel(f4, sheet_name='sample_mut_details_sz', index_col='patient')
df5.head(2)

,HIV status,ARID1A,ATRX,BAZ1A,BAZ1B,BAZ2B,BRD2,BRD3,BRD4,BRD7,...,TBL1X,TFPT,TRIM33,UBE2B,TRRAP,WHSC1L1,YEATS2,YY1,ZMYND11,ZMYND8
patient,,,,,,,,,,,,,,,,,,,,,
HTMCP-03-06-02007,HIV negative,1,1,1,0,1,0,0,1,1,...,0,0,0,0,1,0,0,1,0,1
HTMCP-03-06-02026,HIV negative,1,1,0,1,1,0,1,0,1,...,0,0,1,0,0,0,0,0,1,0


In [156]:
df51 = df5[df5.columns[1:]].T

df51.head(2)

patient,HTMCP-03-06-02007,HTMCP-03-06-02026,HTMCP-03-06-02063,HTMCP-03-06-02214,HTMCP-03-06-02042,HTMCP-03-06-02332,HTMCP-03-06-02185,HTMCP-03-06-02012,HTMCP-03-06-02196,HTMCP-03-06-02203,...,HTMCP-03-06-02076,HTMCP-03-06-02113,HTMCP-03-06-02230,HTMCP-03-06-02170,HTMCP-03-06-02152,HTMCP-03-06-02235,HTMCP-03-06-02195,HTMCP-03-06-02261,HTMCP-03-06-02147,HTMCP-03-06-02267
ARID1A,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATRX,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
df6 = pd.merge(df51, df41, left_index=True, right_index=True)

df6.head(2)

,HTMCP-03-06-02007,HTMCP-03-06-02026,HTMCP-03-06-02063,HTMCP-03-06-02214,HTMCP-03-06-02042,HTMCP-03-06-02332,HTMCP-03-06-02185,HTMCP-03-06-02012,HTMCP-03-06-02196,HTMCP-03-06-02203,...,HTMCP-03-06-02230,HTMCP-03-06-02170,HTMCP-03-06-02152,HTMCP-03-06-02235,HTMCP-03-06-02195,HTMCP-03-06-02261,HTMCP-03-06-02147,HTMCP-03-06-02267,category_1,category_2
ARID1A,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Remodeller,SWI/SNF remodeling complex
ATRX,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Remodeller,SWI/SNF family


In [158]:
def categorize_cols(df, col):
    col_values = df[col].unique()
    lst = []
    for col_value in col_values:
        s = df.query('{0} == "{1}"'.format(col, col_value)).sum()
        lst.append(s)
    dfo = pd.DataFrame(lst)
    dfo.index = col_values
    dfo = dfo.drop([col], axis=1)
    return dfo

In [159]:
df8 = categorize_cols(df6, 'category_1').drop('category_2', axis=1)

df8.head(2)

,HTMCP-03-06-02007,HTMCP-03-06-02026,HTMCP-03-06-02063,HTMCP-03-06-02214,HTMCP-03-06-02042,HTMCP-03-06-02332,HTMCP-03-06-02185,HTMCP-03-06-02012,HTMCP-03-06-02196,HTMCP-03-06-02203,...,HTMCP-03-06-02076,HTMCP-03-06-02113,HTMCP-03-06-02230,HTMCP-03-06-02170,HTMCP-03-06-02152,HTMCP-03-06-02235,HTMCP-03-06-02195,HTMCP-03-06-02261,HTMCP-03-06-02147,HTMCP-03-06-02267
Remodeller,16.0,9.0,5.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Reader,21.0,7.0,6.0,3.0,1.0,1.0,1.0,1.0,2.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [160]:
df9 = categorize_cols(df6, 'category_2').drop('category_1', axis=1)

df9.head(2)

,HTMCP-03-06-02007,HTMCP-03-06-02026,HTMCP-03-06-02063,HTMCP-03-06-02214,HTMCP-03-06-02042,HTMCP-03-06-02332,HTMCP-03-06-02185,HTMCP-03-06-02012,HTMCP-03-06-02196,HTMCP-03-06-02203,...,HTMCP-03-06-02076,HTMCP-03-06-02113,HTMCP-03-06-02230,HTMCP-03-06-02170,HTMCP-03-06-02152,HTMCP-03-06-02235,HTMCP-03-06-02195,HTMCP-03-06-02261,HTMCP-03-06-02147,HTMCP-03-06-02267
SWI/SNF remodeling complex,6.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SWI/SNF family,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [161]:
df10 = pd.concat([df8, df9]).reset_index().dropna().set_index('index').astype(float)

df10.head(2)

,HTMCP-03-06-02007,HTMCP-03-06-02026,HTMCP-03-06-02063,HTMCP-03-06-02214,HTMCP-03-06-02042,HTMCP-03-06-02332,HTMCP-03-06-02185,HTMCP-03-06-02012,HTMCP-03-06-02196,HTMCP-03-06-02203,...,HTMCP-03-06-02076,HTMCP-03-06-02113,HTMCP-03-06-02230,HTMCP-03-06-02170,HTMCP-03-06-02152,HTMCP-03-06-02235,HTMCP-03-06-02195,HTMCP-03-06-02261,HTMCP-03-06-02147,HTMCP-03-06-02267
index,,,,,,,,,,,,,,,,,,,,,
Remodeller,16.0,9.0,5.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Reader,21.0,7.0,6.0,3.0,1.0,1.0,1.0,1.0,2.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
df10[df10 < 0.5] = np.nan
df10[df10 > 0.5] = 'Mutated'
df10.fillna('Wild Type', inplace=True)

In [163]:
df10.head(2)

,HTMCP-03-06-02007,HTMCP-03-06-02026,HTMCP-03-06-02063,HTMCP-03-06-02214,HTMCP-03-06-02042,HTMCP-03-06-02332,HTMCP-03-06-02185,HTMCP-03-06-02012,HTMCP-03-06-02196,HTMCP-03-06-02203,...,HTMCP-03-06-02076,HTMCP-03-06-02113,HTMCP-03-06-02230,HTMCP-03-06-02170,HTMCP-03-06-02152,HTMCP-03-06-02235,HTMCP-03-06-02195,HTMCP-03-06-02261,HTMCP-03-06-02147,HTMCP-03-06-02267
index,,,,,,,,,,,,,,,,,,,,,
Remodeller,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Wild Type,Mutated,Wild Type,Wild Type,...,Wild Type,Wild Type,Wild Type,Wild Type,Wild Type,Wild Type,Wild Type,Wild Type,Wild Type,Wild Type
Reader,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,...,Wild Type,Wild Type,Wild Type,Mutated,Wild Type,Wild Type,Wild Type,Wild Type,Wild Type,Wild Type


In [164]:
df10.shape

(23, 54)

In [165]:
df11 = df10.T

In [166]:
df11.head(2)

index,Remodeller,Reader,Writer,Eraser,SWI/SNF remodeling complex,SWI/SNF family,ISWI remdelling complexes,SEC (Super Elongation) Complex,INO80/SWR remodelling complexes,Testis-specific chromatin protein,...,REST corepressor,ISWI chromatin remdelling,HAT complexes,NuRD complex,compass complex,HDAC complex,GNAT family HAT complexes,MYST family HAT complexes,RNF20/40 complex,TFIID basal transcription factor complex or PCAF histone acetylase complex
HTMCP-03-06-02007,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,...,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated
HTMCP-03-06-02026,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Wild Type,Mutated,Wild Type,...,Wild Type,Wild Type,Mutated,Mutated,Mutated,Mutated,Mutated,Mutated,Wild Type,Mutated


In [167]:
df77 = pd.merge(df11, df66, left_index=True, right_on='patient', how='left').set_index('patient')

In [168]:
df77.head(2)
df77.shape

(54, 27)

In [169]:
f = f'/projects/da_workspace/DA-167_Cervical_epi_clustering/clinic_covariate_20181009.csv'
df77.to_csv(f)